In [1]:
import cv2 , os , ipywidgets , traitlets
import numpy as np
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.usb_camera import USBCamera # USBCamera(width=224, height=224, capture_width=640, capture_height=480 , capture_flip = 0 ,capture_device=0)

In [2]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [3]:
camera = USBCamera()

In [4]:
faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')

In [5]:
def face_recognition(camera_image):
    face_image = np.copy(camera_image)
    gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        
        scaleFactor=1.2,
        minNeighbors=5
        ,     
        minSize=(20, 20)
    )
    
    for (x,y,w,h) in faces:
        face_image = cv2.rectangle(face_image,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = face_image[y:y+h, x:x+w]
        
    jpeg_image = bgr8_to_jpeg(face_image)
    return jpeg_image

In [6]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=face_recognition)
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [7]:
camera.running = True

def update_image(change):
    image = change['new']
    
camera.observe(update_image, names='value')

In [ ]:
camera.running = False
camera.unobserve(update_image, names='value')